### BentoML Example
# Titanic Survival Prediction with LightBGM

This is a BentoML Demo Project demonstrating how to package and serve LightBGM model for production using BentoML.

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment.

Let's get started!
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=ligthbgm&ea=lightbgm-tiantic-survival-prediction&dt=lightbgm-tiantic-survival-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install bentoml
!pip install lightgbm numpy pandas

In [4]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import bentoml

# Prepare Dataset
download dataset from https://www.kaggle.com/c/titanic/data

In [3]:
!mkdir data
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv -o ./data/train.csv
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv -o ./data/test.csv

mkdir: data: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60302  100 60302    0     0   154k      0 --:--:-- --:--:-- --:--:--  154k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28210  100 28210    0     0   101k      0 --:--:-- --:--:-- --:--:--  101k


In [5]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
y = train_df.pop('Survived')
cols = ['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']
X_train, X_test, y_train, y_test = train_test_split(train_df[cols], 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [7]:
# Create an LGBM dataset for training
train_data = lgb.Dataset(data=X_train[cols],
                        label=y_train)

# Create an LGBM dataset from the test
test_data = lgb.Dataset(data=X_test[cols],
                        label=y_test)

# Model Training

In [8]:
lgb_params = {
    'boosting': 'dart',          # dart (drop out trees) often performs better
    'application': 'binary',     # Binary classification
    'learning_rate': 0.05,       # Learning rate, controls size of a gradient descent step
    'min_data_in_leaf': 20,      # Data set is quite small so reduce this a bit
    'feature_fraction': 0.7,     # Proportion of features in each boost, controls overfitting
    'num_leaves': 41,            # Controls size of tree since LGBM uses leaf wise splits
    'metric': 'binary_logloss',  # Area under ROC curve as the evaulation metric
    'drop_rate': 0.15
              }

evaluation_results = {}
model = lgb.train(train_set=train_data,
                params=lgb_params,
                valid_sets=[train_data, test_data], 
                valid_names=['Train', 'Test'],
                evals_result=evaluation_results,
                num_boost_round=500,
                early_stopping_rounds=100,
                verbose_eval=20
                )

[20]	Train's binary_logloss: 0.55215	Test's binary_logloss: 0.587358
[40]	Train's binary_logloss: 0.510164	Test's binary_logloss: 0.559348
[60]	Train's binary_logloss: 0.500602	Test's binary_logloss: 0.551635
[80]	Train's binary_logloss: 0.490215	Test's binary_logloss: 0.547154
[100]	Train's binary_logloss: 0.486812	Test's binary_logloss: 0.547076
[120]	Train's binary_logloss: 0.479242	Test's binary_logloss: 0.542552
[140]	Train's binary_logloss: 0.469847	Test's binary_logloss: 0.539319
[160]	Train's binary_logloss: 0.471384	Test's binary_logloss: 0.542278
[180]	Train's binary_logloss: 0.453052	Test's binary_logloss: 0.535512
[200]	Train's binary_logloss: 0.442048	Test's binary_logloss: 0.533921
[220]	Train's binary_logloss: 0.436788	Test's binary_logloss: 0.534261
[240]	Train's binary_logloss: 0.427196	Test's binary_logloss: 0.532026
[260]	Train's binary_logloss: 0.420145	Test's binary_logloss: 0.531791
[280]	Train's binary_logloss: 0.413336	Test's binary_logloss: 0.527412
[300]	Train

In [9]:
test_df['pred'] = model.predict(test_df[cols])
test_df[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)

,Pclass,Age,Fare,SibSp,Parch,pred
10,3,NaN,7.8958,0,0,0.052353
11,1,46.0,26.0000,0,0,0.308877


## Create BentoService for model serving

In [10]:
%%writefile lightbgm_titanic_bento_service.py

import lightgbm as lgb

import bentoml
from bentoml.artifact import LightGBMModelArtifact
from bentoml.adapters import DataframeInput

@bentoml.artifacts([LightGBMModelArtifact('model')])
@bentoml.env(pip_dependencies=['lightgbm'])
class TitanicSurvivalPredictionService(bentoml.BentoService):
    
    @bentoml.api(input=DataframeInput())
    def predict(self, df):
        data = df[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']]
        return self.artifacts.model.predict(data)

Overwriting lightbgm_titanic_bento_service.py


# Save BentoML service archive

In [11]:
# 1) import the custom BentoService defined above
from lightbgm_titanic_bento_service import TitanicSurvivalPredictionService

# 2) `pack` it with required artifacts
bento_service = TitanicSurvivalPredictionService()
bento_service.pack('model', model)

# 3) save your BentoSerivce
saved_path = bento_service.save()

[2020-08-04 10:02:12,116] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-08-04 10:02:22,865] INFO - Detect BentoML installed in development model, copying local BentoML module file to target saved bundle path
running sdist
running egg_info
writing BentoML.egg-info/PKG-INFO
writing dependency_links to BentoML.egg-info/dependency_links.txt
writing entry points to BentoML.egg-info/entry_points.txt
writing requirements to BentoML.egg-info/requires.txt
writing top-level names to BentoML.egg-info/top_level.txt
reading manifest file 'BentoML.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'


no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.8.3+47.g5daa71b
creating BentoML-0.8.3+47.g5daa71b/BentoML.egg-info
creating BentoML-0.8.3+47.g5daa71b/bentoml
creating BentoML-0.8.3+47.g5daa71b/bentoml/adapters
creating BentoML-0.8.3+47.g5daa71b/bentoml/artifact
creating BentoML-0.8.3+47.g5daa71b/bentoml/cli
creating BentoML-0.8.3+47.g5daa71b/bentoml/clipper
creating BentoML-0.8.3+47.g5daa71b/bentoml/configuration
creating BentoML-0.8.3+47.g5daa71b/bentoml/configuration/__pycache__
creating BentoML-0.8.3+47.g5daa71b/bentoml/handlers
creating BentoML-0.8.3+47.g5daa71b/bentoml/marshal
creating BentoML-0.8.3+47.g5daa71b/bentoml/saved_bundle
creating BentoML-0.8.3+47.g5daa71b/bentoml/server
creating BentoML-0.8.3+47.g5daa71b/bentoml/utils
creating BentoML-0.8.3+47.g5daa71b/bentoml/yatai
creating BentoML-0.8.3+47.g5daa71b/bentoml/yatai/client
creating BentoML-0.8.3+47.g5daa71b/bentoml/yatai/deployment
creating BentoML-0.8.3+47.g5daa71b/bentoml/yatai/dep

## Load saved BentoService for serving


In [12]:
import bentoml

bento_model = bentoml.load(saved_path)

result = bento_model.predict(test_df)
test_df['pred'] = result
test_df[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)

[2020-08-04 10:02:31,584] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.3, but loading from BentoML version 0.8.3+47.g5daa71b
[2020-08-04 10:02:31,585] WARNING - Module `lightbgm_titanic_bento_service` already loaded, using existing imported module.
[2020-08-04 10:02:31,587] WARNING - `load` on a 'packed' artifact may lead to unexpected behaviors
[2020-08-04 10:02:31,622] WARNING - `pack` an artifact multiple times may lead to unexpected behaviors


,Pclass,Age,Fare,SibSp,Parch,pred
10,3,NaN,7.8958,0,0,0.052353
11,1,46.0,26.0000,0,0,0.308877


# Access BentoService bundle with CLI

In [ ]:
!bentoml get TitanicSurvivalPredictionService

In [18]:
!bentoml get {bento_service.name}:{bento_service.version}

{
  "name": "TitanicSurvivalPredictionService",
  "version": "20200804100212_2598E6",
  "uri": {
    "type": "LOCAL",
    "uri": "/home/bentoml/bentoml/repository/TitanicSurvivalPredictionService/20200804100212_2598E6"
  },
  "bentoServiceMetadata": {
    "name": "TitanicSurvivalPredictionService",
    "version": "20200804100212_2598E6",
    "createdAt": "2020-08-04T02:02:23.770916Z",
    "env": {
      "condaEnv": "name: bentoml-TitanicSurvivalPredictionService\nchannels:\n- defaults\ndependencies:\n- python=3.6.10\n- pip\n",
      "pipDependencies": "lightgbm\npandas\nbentoml==0.8.3",
      "pythonVersion": "3.6.10",
      "dockerBaseImage": "bentoml/model-server:0.8.3"
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "LightGBMModelArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "inputType": "DataframeInput",
        "docs": "BentoService inference API 'predict', input: 'DataframeInput', output: 'DefaultOutput'",
   

In [20]:
!bentoml run TitanicSurvivalPredictionService:latest predict \
--input '{"PassengerId":{"3":895},"Pclass":{"3":3},"Name":{"3":"Wirz, Mr. Albert"},"Sex":{"3":"male"},"Age":{"3":27.0},"SibSp":{"3":0},"Parch":{"3":0},"Ticket":{"3":"315154"},"Fare":{"3":8.6625},"Cabin":{"3":null},"Embarked":{"3":"S"},"pred":{"3":0.5045963287}}'

[2020-08-04 10:04:51,776] INFO - Getting latest version TitanicSurvivalPredictionService:20200804100212_2598E6
[2020-08-04 10:04:52,809] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-08-04 10:04:52,836] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.3, but loading from BentoML version 0.8.3+47.g5daa71b
[0.50459633]


## BentoService Serving via REST API

In your termnial, run the following command to start the REST API server:

In [22]:
!bentoml serve TitanicSurvivalPredictionService:latest --enable-microbatch

[2020-08-04 10:05:18,992] INFO - Getting latest version TitanicSurvivalPredictionService:20200804100212_2598E6
[2020-08-04 10:05:18,992] INFO - Starting BentoML API server in development mode..
[2020-08-04 10:05:19,966] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-08-04 10:05:19,988] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.3, but loading from BentoML version 0.8.3+47.g5daa71b
[2020-08-04 10:05:21,477] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.3, but loading from BentoML version 0.8.3+47.g5daa71b
[2020-08-04 10:05:21,478] INFO - Micro batch enabled for API `predict`
[2020-08-04 10:05:21,480] INFO - Your system nofile limit is 10000, which means each in

Open http://127.0.0.1:5000 to see more information about the REST APIs server in your
browser.


### Send prediction requeset to the REST API server

Navigate to parent directory of the notebook(so you have reference to the `test.jpg` image), and run the following `curl` command to send the image to REST API server and get a prediction result:

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' \
localhost:5000/predict
```

## Use BentoService as PyPI package

In [23]:
!pip install --quiet {saved_path}

In [34]:
!TitanicSurvivalPredictionService info

[2020-01-22 18:21:12,485] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-22 18:21:12,494] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.8,  but loading from BentoML version 0.5.8+23.g1dd72d3
{
  "name": "TitanicSurvivalPredictionService",
  "version": "20200122154116_FCEE7A",
  "created_at": "2020-01-22T23:41:33.581979Z",
  "env": {
    "conda_env": "name: bentoml-TitanicSurvivalPredictionService\nchannels:\n- defa

In [25]:
!TitanicSurvivalPredictionService run predict --input '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]'

[0.80731289]


# Containerize REST API server with Docker


The BentoService SavedBundle is structured to work as a docker build context, that can be directed used to build a docker image for API server. Simply use it as the docker build context directory:

In [26]:
!docker build --quiet -t lightgbm-titanic {saved_path}

sha256:cc5736d088e4beea88863682c5107dd4d2a8e067670db1cf667134569936896f


Next, you can docker push the image to your choice of registry for deployment, or run it locally for development and testing:

In [28]:
!docker run -p 5000:5000 lightgbm-titanic --enable-microbatch

[2020-08-04 02:09:42,448] INFO - Starting BentoML API server in production mode..
[2020-08-04 02:09:42,880] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-08-04 02:09:42,890] INFO - Running micro batch service on :5000
[2020-08-04 02:09:42 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-08-04 02:09:42 +0000] [12] [INFO] Starting gunicorn 20.0.4
[2020-08-04 02:09:42 +0000] [1] [INFO] Listening at: http://0.0.0.0:40927 (1)
[2020-08-04 02:09:42 +0000] [12] [INFO] Listening at: http://0.0.0.0:5000 (12)
[2020-08-04 02:09:42 +0000] [12] [INFO] Using worker: aiohttp.worker.GunicornWebWorker
[2020-08-04 02:09:42 +0000] [1] [INFO] Using worker: sync
[2020-08-04 02:09:42 +0000] [13] [INFO] Booting worker with pid: 13
[2020-08-04 02:09:42 +0000] [14] [INFO] Booting worker with pid: 14
[2020-08-04 02:09:42 +0000] [15] [INFO] Booting worker with pid: 15
[2020-08-04 02:09:42,920] WARNING - Using BentoML not from official PyPI release. In order to find the same version of Bento

# Deploy BentoService as REST API server to the cloud


BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html

For this demo, we are going to deploy to AWS Sagemaker

**Deploying to Sagemaker with `bentoml sagemaker deploy` command**

In [24]:
!bentoml sagemaker deploy first-light-bgm -b {bento_service.name}:{bento_service.version} \
    --api-name predict --region us-west-2

[2020-01-22 16:59:41,021] DEBUG - Using BentoML with local Yatai server
[2020-01-22 16:59:41,106] DEBUG - Upgrading tables to the latest revision
Deploying Sagemaker deployment /[2020-01-22 16:59:42,117] DEBUG - Created temporary directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-esrezr4x
\[2020-01-22 16:59:42,748] DEBUG - Getting docker login info from AWS
[2020-01-22 16:59:42,749] DEBUG - Building docker image: 192023623294.dkr.ecr.us-west-2.amazonaws.com/titanicsurvivalpredictionservice-sagemaker:20200122154116_FCEE7A
-[2020-01-22 16:59:43,159] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-01-22 16:59:43,159] INFO - 

[2020-01-22 16:59:43,160] INFO -  ---> 406f2b43ea59

[2020-01-22 16:59:43,160] INFO - Step 2/11 : EXPOSE 8080
[2020-01-22 16:59:43,160] INFO - 

[2020-01-22 16:59:43,160] INFO -  ---> Using cache

[2020-01-22 16:59:43,160] INFO -  ---> d8ec94af8603

[2020-01-22 16:59:43,160] INFO - Step 3/11 : RUN set -x      && apt-get upd

** Use `bentoml sagemaker get DEPLOYMENT_NAME` to get latest state and other information such as endpoint**

In [25]:
!bentoml sagemaker get first-light-bgm

{
  "namespace": "dev",
  "name": "first-light-bgm",
  "spec": {
    "bentoName": "TitanicSurvivalPredictionService",
    "bentoVersion": "20200122154116_FCEE7A",
    "operator": "AWS_SAGEMAKER",
    "sagemakerOperatorConfig": {
      "region": "us-west-2",
      "instanceType": "ml.m4.xlarge",
      "instanceCount": 1,
      "apiName": "predict"
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "EndpointName": "dev-first-light-bgm",
      "EndpointArn": "arn:aws:sagemaker:us-west-2:192023623294:endpoint/dev-first-light-bgm",
      "EndpointConfigName": "dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A",
      "ProductionVariants": [
        {
          "VariantName": "dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A",
          "DeployedImages": [
            {
              "SpecifiedImage": "192023623294.dkr.ecr.us-west-2.amazonaws.com/titanicsurvivalpredictionservice-sagemaker:20200122154116_FCEE7A",
              "ResolvedImage": "192023

Test our sagemaker deployment with a sample data

In [29]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name dev-first-light-bgm \
  --body '{"PassengerId":{"3":895},"Pclass":{"3":3},"Name":{"3":"Wirz, Mr. Albert"},"Sex":{"3":"male"},"Age":{"3":27.0},"SibSp":{"3":0},"Parch":{"3":0},"Ticket":{"3":"315154"},"Fare":{"3":8.6625},"Cabin":{"3":null},"Embarked":{"3":"S"},"pred":{"3":0.5045963287}}' \
  --content-type "application/json" \
  output.json && cat output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "dev-first-light--TitanicSurvivalPredi-20200122154116-FCEE7A"
}
[0.5045963286563061]

**`bentoml sagemaker list` command provides an overview of all of the Sagemaker deployments**

In [30]:
!bentoml sagemaker list

NAME             NAMESPACE    LABELS    PLATFORM       STATUS    AGE
first-light-bgm  dev                    aws-sagemaker  running   14 minutes and 37 seconds


**`bentoml sagemaker delete` deletes the sagemaker deployment and related resources on AWS**

In [31]:
!bentoml sagemaker delete first-light-bgm

Successfully deleted AWS Sagemaker deployment "first-light-bgm"
